This code is a Python script for implementing a basic trading bot that uses the Moving Average Convergence Divergence (MACD) indicator as part of its trading strategy. Here's a breakdown of its functionality and the financial concepts involved:

Imports and Setup: The script imports necessary libraries such as ccxt for interacting with cryptocurrency exchanges, pandas for data manipulation, logging for logging events, datetime for handling date and time, and smtplib for sending email alerts. It also initializes logging configuration and defines a function to send email alerts.

Exchange Configuration: It sets up the exchange (in this case, WazirX) with API key and secret for accessing trading functionalities.

Data Retrieval: The script fetches historical price data for two cryptocurrency pairs: XMR/BTC (Monero to Bitcoin) and XMR/USDT (Monero to Tether, a stablecoin pegged to the US dollar).

Data Processing: It converts the fetched data into pandas DataFrame and calculates the MACD indicator for both pairs. MACD is a trend-following momentum indicator that shows the relationship between two moving averages of an asset's price.

Risk Management: The script calculates the position size based on the trader's account balance, risk per trade, and stop loss level. Position size determines how much of a cryptocurrency to buy or sell in each trade.

Execution Logic: It defines a function to execute trades based on the trading signals generated by the MACD strategy. The script sends email alerts for successful trades and errors.

MACD Trading Strategy: The main trading strategy is based on the MACD indicator. When the MACD line crosses above the Signal line, it generates a buy signal, indicating a potential uptrend. Conversely, when the MACD line crosses below the Signal line, it generates a sell signal, indicating a potential downtrend.

Position Sizing: It calculates the position size based on the trader's risk management rules. Position sizing helps control the amount of capital exposed to each trade and ensures consistent risk management.

Execution: The script applies the MACD trading strategy to the XMR/BTC and XMR/USDT pairs, executing trades based on the generated signals.

Overall, this script automates the process of cryptocurrency trading based on a simple MACD trading strategy. It demonstrates the integration of technical analysis indicators into an algorithmic trading bot for making buy and sell decisions in the cryptocurrency markets. However, it's essential to note that trading involves risks, and past performance is not indicative of future results. It's crucial for traders to thoroughly backtest their strategies and exercise caution when deploying automated trading bots in live markets.

In [5]:
import ccxt
import pandas as pd
import logging
from datetime import datetime
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# Set up logging
logging.basicConfig(filename='trading_bot.log', level=logging.INFO, format='%(asctime)s %(message)s')

In [1]:
# Function to send email alerts
def send_email_alert(subject, message):
    sender_email = "your_email@example.com"
    receiver_email = "recipient@example.com"
    smtp_server = "smtp.example.com"
    smtp_port = 587  # Change the port if required
    smtp_username = "your_smtp_username"
    smtp_password = "your_smtp_password"

    # Create message container
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject

    # Add message body
    msg.attach(MIMEText(message, 'plain'))

    # Start SMTP session
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(smtp_username, smtp_password)
        server.send_message(msg)

In [8]:
# Initialize the exchange
exchange = ccxt.wazirx({
    'apiKey': 'YOUR_API_KEY',
    'secret': 'YOUR_SECRET',
    'enableRateLimit': True,
})

def fetch_data(symbol):
    try:
        data = exchange.fetch_ohlcv(symbol, timeframe='1d', limit=200)
        logging.info(f'Successfully fetched historical price data for {symbol}.')
        return data
    except Exception as e:
        logging.error(f"An error occurred while fetching data: {e}")
        return None

In [ ]:
# Fetch historical price data for XMR/BTC and XMR/USDT pairs
xmr_btc_data = fetch_data('XMR/BTC')
xmr_usdt_data = fetch_data('XMR/USDT')

if xmr_btc_data is None or xmr_usdt_data is None:
    logging.error("Failed to fetch data. Exiting.")
    exit(1)


In [ ]:
# Convert fetched data to pandas DataFrame
def ohlcv_to_dataframe(ohlcv_data):
    df = pd.DataFrame(ohlcv_data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    return df

xmr_btc_df = ohlcv_to_dataframe(xmr_btc_data)
xmr_usdt_df = ohlcv_to_dataframe(xmr_usdt_data)

In [ ]:
# Calculate MACD for XMR/BTC and XMR/USDT pairs
def calculate_macd(df, short_window=12, long_window=26, signal_window=9):
    df['ShortEMA'] = df['close'].ewm(span=short_window, min_periods=1, adjust=False).mean()
    df['LongEMA'] = df['close'].ewm(span=long_window, min_periods=1, adjust=False).mean()
    df['MACD'] = df['ShortEMA'] - df['LongEMA']
    df['Signal'] = df['MACD'].ewm(span=signal_window, min_periods=1, adjust=False).mean()
    return df

xmr_btc_df = calculate_macd(xmr_btc_df)
xmr_usdt_df = calculate_macd(xmr_usdt_df)

In [ ]:
# Risk Management
def calculate_position_size(account_balance, risk_per_trade, stop_loss):
    position_size = (account_balance * risk_per_trade) / stop_loss
    return position_size

In [ ]:
# Execution Logic
def execute_trade(exchange, symbol, type, side, amount, price):
    try:
        order = exchange.create_order(symbol, type, side, amount, price)
        logging.info(f"Successfully executed {side} order for {symbol}.")
        send_email_alert("Trade Executed", f"Trade executed for {symbol}. Side: {side}, Amount: {amount}, Price: {price}")
        return order
    except Exception as e:
        logging.error(f"An error occurred while executing trade: {e}")
        send_email_alert("Trade Execution Error", f"Error occurred while executing trade for {symbol}. Error: {e}")
        return None

In [ ]:
# Position Sizing
account_balance = 10000  # Your account balance
risk_per_trade = 0.01  # Risk 1% per trade
stop_loss = 50  # Stop loss in $
position_size = calculate_position_size(account_balance, risk_per_trade, stop_loss)

In [ ]:
# Define MACD trading strategy
def macd_trading_strategy(df, symbol):
    positions = []
    for i in range(len(df)):
        # Buy signal: MACD crosses above Signal line
        if df['MACD'].iloc[i] > df['Signal'].iloc[i] and df['MACD'].iloc[i - 1] <= df['Signal'].iloc[i - 1]:
            position = 'buy'
            order = execute_trade(exchange, symbol, 'market', 'buy', position_size, None)
            if order is not None:
                positions.append((df.index[i], position, order))
        # Sell signal: MACD crosses below Signal line
        elif df['MACD'].iloc[i] < df['Signal'].iloc[i] and df['MACD'].iloc[i - 1] >= df['Signal'].iloc[i - 1]:
            position = 'sell'
            order = execute_trade(exchange, symbol, 'market', 'sell', position_size, None)
            if order is not None:
                positions.append((df.index[i], position, order))
    return positions


In [ ]:
# Apply MACD trading strategy to XMR/BTC and XMR/USDT pairs
xmr_btc_positions = macd_trading_strategy(xmr_btc_df, 'XMR/BTC')
xmr_usdt_positions = macd_trading_strategy(xmr_usdt_df, 'XMR/USDT')

In [ ]:
# Display trading positions
print("XMR/BTC Trading Positions:")
for position in xmr_btc_positions:
    print(f"Date: {position[0]}, Position: {position[1]}, Order: {position[2]}")

print("\nXMR/USDT Trading Positions:")
for position in xmr_usdt_positions:
    print(f"Date: {position[0]}, Position: {position[1]}, Order: {position[2]}")

In [1]:
# In this code:
# I added the send_email_alert() function to send email alerts for trade executions and errors.
# Modified the execute_trade() function to trigger email alerts upon trade execution or error.
# Integrated email alerts into the trading strategy logic to notify you whenever a trade is executed or when an error occurs during trade execution.

# Here's a list of parameters and variables that you need to customize with your own values for the trading bot to work properly:
# API Key and Secret: Replace 'YOUR_API_KEY' and 'YOUR_SECRET' with your actual API key and secret provided by the exchange you are using.
# Email Configuration:
# Replace "your_email@example.com" with your email address for sending alerts.
# Replace "recipient@example.com" with the recipient's email address.
# Replace "smtp.example.com" with your SMTP server address.
# Modify smtp_port, smtp_username, and smtp_password according to your SMTP server configuration.
# Trading Pair: Set the trading pairs you want to trade with. For example, if you want to trade XMR/BTC and XMR/USDT, you can keep 'XMR/BTC' and 'XMR/USDT'.
# Account Balance: Set account_balance to your actual account balance.
# Risk per Trade: Set risk_per_trade to the percentage of your account balance you are willing to risk per trade.
# Stop Loss: Set stop_loss to the desired stop loss amount in the quote currency (e.g., USD for XMR/USDT).
# SMTP Server Configuration: Ensure that the SMTP server details (smtp_server, smtp_port, smtp_username, and smtp_password) are correctly configured according to your email service provider.
# Trade Execution Parameters: If needed, you can adjust parameters such as order type ('market', 'limit', etc.), order side ('buy', 'sell'), and order amount (position_size).

# After customizing these parameters with your actual values, you can run the code to start executing trades based on your MACD strategy and receive email alerts for trade placements and executions.